In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-l9h_8c8b
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-l9h_8c8b
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 881.9/881.9 MB 177.3 MB/s eta 0:0

In [3]:
# gpu 프리미엄 사용 시
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 2.0/2.0 GB 43.6 MB/s eta 0:00:01tcmalloc: large alloc 2041339904 bytes == 0x3612000 @  0x7f0d7149b1e7 0x4d30a0 0x4d312c 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2
tcmalloc: large alloc 2551676928 bytes == 0x7d0d8000 @  0x7f0d7149c615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941
tcmalloc: large alloc 2041339904 bytes == 0x3612000 @  0x7f0d7149b1e7 0x4

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from sklearn.model_selection import train_test_split

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import pandas as pd
from tqdm.notebook import tqdm

import re

In [5]:
##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

from transformers import AutoTokenizer, AutoModelForSequenceClassification

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


### 데이터 전처리


In [6]:
import pandas as pd

# 데이터 입력


In [7]:
df = pd.read_csv('/content/drive/MyDrive/뉴스원인분석_modeling/data/kia_newslabel_text_rank_max_kss_1230.csv')
df = df.rename(columns={'date_cate' : 'date', 'standeard_tagging0.25' : 'target'})
df

,date,content,standeard_tagging0.5,전일비_비율,standeard_tagging0.3,target,targetpm,text_rank_content,len,text_rank_len,mpc_pos,mpc_neg,mpck_token,score_diff,score_diff+tar,pos_neg,pos_neg_tardiff,kmm_label,kmm_label_n5,kmm_label_n_n5
0,2017-12-04,현대기아차 북미시장 고전 지속코나스팅어 기대로스앤젤레스연합뉴스 옥철 특파원 현대차...,2,-0.15,2,2,1,"현대차는 오토쇼에 데뷔한 콤팩트 스포츠유틸리티차 코나ona, 기아차는 2018 ...",958,288,9.960439e-01,0.003956,현대기아차 북미시장 고전 지속코나스팅어 기대로스앤젤레스연합뉴스 옥철 특파원 현대차...,0.992088,2.492088,pos,pos,2,매우상승,4
1,2017-12-04,기업공시12월 1일서울경제 대표이사 손봉영에서 김종호로 변경 인터플렉스 유상증자에...,2,-0.15,2,2,1,기업공시12월 1일서울경제 대표이사 손봉영에서 김종호로 변경 인터플렉스 유상증자에...,1372,473,3.168241e-06,0.999997,기업공시12월 1일서울경제 대표이사 손봉영에서 김종호로 변경 인터플렉스 유상증자에...,-0.999994,0.500006,neg,pos,4,하락,1
2,2017-12-04,뒷좌석 어린이 안전도 평가카시트 장착한 6세 아동 충격 더 크다헤럴드경제박도제 기자...,2,-0.15,2,2,1,뒷좌석 어린이 안전도 평가카시트 장착한 6세 아동 충격 더 크다헤럴드경제박도제 기자...,1618,392,1.344403e-05,0.999987,뒷좌석 어린이 안전도 평가카시트 장착한 6세 아동 충격 더 크다헤럴드경제박도제 기자...,-0.999973,0.500027,neg,pos,4,하락,1
3,2017-12-04,"국산차, 연말 파격 할인싼타페 230만원캡티바 500만원이데일리 김보경 기자 국내 ...",2,-0.15,2,2,1,한국은 12월 한 달 동안 전 차종에 대해 515의 할인을 적용한다. 주력 차...,1014,237,3.493616e-05,0.999965,"국산차, 연말 파격 할인싼타페 230만원캡티바 500만원이데일리 김보경 기자 국내 ...",-0.999930,0.500070,neg,pos,4,하락,1
4,2017-12-04,상장 대목이번주 진에어 등 7개 증시 입성서울연합인포맥스 김경림 기자 이번 주4일...,2,-0.15,2,2,1,진에어는 공모청약 경쟁률 134. 동양피스톤은 내연기관용 피스톤을 만드는 회사로 ...,722,133,1.425929e-01,0.857407,상장 대목이번주 진에어 등 7개 증시 입성서울연합인포맥스 김경림 기자 이번 주4일...,-0.714814,0.785186,neg,pos,3,중간,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39905,2022-08-22,"코스피 장중 2,450대로 추락환율, 급등하며 1,340원 근접종합서울연합뉴스 이미...",2,0.26,2,2,3,"지수는 전장보다 25. 이날 서울 외환시장에서 원달러 환율은 장중 1,330원을 ...",1729,86,1.768957e-07,1.000000,"코스피 장중 2,450대로 추락환율, 급등하며 1,340원 근접종합서울연합뉴스 이미...",-1.000000,0.500000,neg,pos,4,하락,1
39906,2022-08-22,기아의 유럽 전략형 모델 씨드서울연합뉴스 기아의 유럽 맞춤형 전략 차종인 씨드eed...,2,0.26,2,2,3,기아의 유럽 전략형 모델 씨드서울연합뉴스 기아의 유럽 맞춤형 전략 차종인 씨드eed...,89,54,5.642329e-01,0.435767,기아의 유럽 전략형 모델 씨드서울연합뉴스 기아의 유럽 맞춤형 전략 차종인 씨드eed...,0.128466,1.628466,pos,pos,1,상승,3
39907,2022-08-22,"현대차그룹엔솔 인니 배터리셀 합작공장, 약 1조원 투자금 확보서울연합뉴스 최평천 기...",2,0.26,2,2,3,"현대차그룹엔솔 인니 배터리셀 합작공장, 약 1조원 투자금 확보서울연합뉴스 최평천 기...",907,456,1.326228e-07,1.000000,"현대차그룹엔솔 인니 배터리셀 합작공장, 약 1조원 투자금 확보서울연합뉴스 최평천 기...",-1.000000,0.500000,neg,pos,4,하락,1
39908,2022-08-22,"미, 탈레반과 아프간 동결 자산 해제 협상 계속 추진서울연합뉴스 김정은 기자 미국...",2,0.26,2,2,3,미국은 탈레반이 지난해 8월 친미 아프간 정부를 무너뜨리고 아프간을 장악하자 미국...,983,376,1.319802e-05,0.999987,"미, 탈레반과 아프간 동결 자산 해제 협상 계속 추진서울연합뉴스 김정은 기자 미국...",-0.999974,0.500026,neg,pos,4,하락,1


In [10]:
df_gr = df.groupby('date')[['target', 'targetpm']].mean()
# 날짜별로 train_test_split
train_date, test_date = train_test_split(df_gr, test_size=0.1, random_state=2022)
train_date_list = train_date.index
test_date_list = test_date.index
train_data = df[df['date'].isin(train_date_list)][['mpck_token', 'target', 'date','targetpm']]
test_data = df[df['date'].isin(test_date_list)][['mpck_token', 'target', 'date','targetpm']]
train_data.shape, test_data.shape

((35897, 4), (3933, 4))

In [11]:
df['target'].value_counts()

1    11645
2    11242
3     8930
4     4079
0     4014
Name: target, dtype: int64

In [12]:
df['targetpm'].value_counts()

3    21259
1    18651
Name: targetpm, dtype: int64

In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [14]:
max_len = 128 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 50
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [25]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [17]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [18]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [20]:
model= torch.load("/content/drive/MyDrive/뉴스원인분석_modeling/model/KoBERT_lmodel_m128_bat64_ep50_1230.pt")
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [21]:
test_data

,mpck_token,target,date,targetpm
175,한기평 내년 자동차업황 비우호적미중 시장 주목서울연합인포맥스 이민재 기자 국내 자...,2,2017-12-12,3
176,"이브자리, 연말 사랑의 이불 1000채 기부이데일리 박경훈 기자 침구전문업체 이브자...",2,2017-12-12,3
192,"단독 이노션, 광고대행사 인수 잰걸음현대자동차그룹 계열 광고대행사인 이노션이 기아...",2,2017-12-12,3
197,"김승연, 한화큐셀 공장 방문세계 1등 지위 강화치둥공장, 미래 태양광 사업 이끄는...",2,2017-12-12,3
198,광주 서구의회 기아차 공장 주변 악취 원인 규명해야광주연합뉴스 장아름 기자 광주 ...,2,2017-12-12,3
...,...,...,...,...
39668,세련되게 잘 빠졌네 [SEP] 패밀리카 뺨치는 가성비쌍용자동차의 토레스는 지난해 렌...,2,2022-08-09,3
39670,엄마 마음 훔치더니.. [SEP] 당당하게 가족차 된 셀토스 시승기국내 소형 스포츠...,2,2022-08-09,3
39671,"서머 정규 리그 막바지젠지, 1위 확정까지 단 1승리그오브레전드챔피언스코리아대표 오...",2,2022-08-09,3
39672,"지바이크, 성남 킥보드 인수전동킥보드 공유 서비스 플랫폼 지쿠터를 운영 중인 지바이...",2,2022-08-09,3


In [26]:
# batch_size 수정 코드
batch_size = 128

test_data3 = test_data.copy()
test_data3 = test_data3.reset_index(drop=True )
test_data3['pred'] = ''

model.eval()
unseen_values = test_data3[['mpck_token', 'target']].values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)
for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  for outn in range(len(out)) :
    pred = int(out[outn].argmax().cpu().numpy())
    idx = batch_size * batch_id + outn
    test_data3.iloc[idx, -1] = pred
    if idx % 5000 == 0 :
      print(idx, pred)
test_data3.head(20)

0it [00:00, ?it/s]

0 1


,mpck_token,target,date,targetpm,pred
0,한기평 내년 자동차업황 비우호적미중 시장 주목서울연합인포맥스 이민재 기자 국내 자...,2,2017-12-12,3,1
1,"이브자리, 연말 사랑의 이불 1000채 기부이데일리 박경훈 기자 침구전문업체 이브자...",2,2017-12-12,3,1
2,"단독 이노션, 광고대행사 인수 잰걸음현대자동차그룹 계열 광고대행사인 이노션이 기아...",2,2017-12-12,3,1
3,"김승연, 한화큐셀 공장 방문세계 1등 지위 강화치둥공장, 미래 태양광 사업 이끄는...",2,2017-12-12,3,1
4,광주 서구의회 기아차 공장 주변 악취 원인 규명해야광주연합뉴스 장아름 기자 광주 ...,2,2017-12-12,3,1
5,"자유로 연비 르노삼성 6 e, 자유로를 달리다이데일리 오토in 김학수 기자 르노삼성...",2,2017-12-12,3,1
6,"노블클라쎄, 합리적 프리미엄 내세운 고급 미니밴노블클라쎄 카니발 9 국내 유일의 프...",2,2017-12-12,3,1
7,한화큐셀 공장 찾은 김승연 태양광은 한화의 미래인수후 7년만에 치둥공장 첫 방문 글...,2,2017-12-12,3,1
8,"사드 피해 보듬고 기업 살리자..., 현대차 충칭공장 간다대통령 오늘 방중 16일 ...",2,2017-12-12,3,1
9,"시승기스팅어 3가지 매력매혹적 디자인주행성능, 그리고 가격까지헤럴드경제배두헌 기자 ...",2,2017-12-12,3,1


In [29]:
# 뉴스별 라벨링
# 뉴스 빈도 + 날 빈도
test_data4 = test_data.copy()
test_data4['pred'] = test_data3['pred'].values 
test_data4['pred'] = test_data4['pred'].astype('int')

df_gr = test_data4.groupby('date')[['target', 'pred', 'targetpm']].mean()
df_date = df_gr.index
for idx in df_date:
  df_gr.loc[idx, 'cnt_pred'] = int(test_data4[test_data4['date'] == idx ]['pred'].value_counts().idxmax())

df_gr['pred_label'] = df_gr['cnt_pred'].apply(lambda x : int(np.round(x)))
df_gr['target'] = df_gr['target'].apply(lambda x : int(np.round(x)))
df_gr['pred_pm'] = df_gr['pred_label'].apply(lambda x: 1 if x< 2 else 3)


df_gr = df_gr.reset_index()
for idx in df_gr.index :
  if df_gr.loc[idx, 'target'] == df_gr.loc[idx, 'pred_label']:
    temp_eval = True
  else :
    temp_eval = False
  df_gr.loc[idx, 'eval_5'] = temp_eval

for idx in df_gr.index :
  if df_gr.loc[idx, 'target'] == 0 :
    df_gr.loc[idx, 'target_3label'] = 1
  elif df_gr.loc[idx, 'target'] == 4 :
    df_gr.loc[idx, 'target_3label'] = 3
  else :
    df_gr.loc[idx, 'target_3label'] = int(df_gr.loc[idx, 'target'])

  if df_gr.loc[idx, 'pred_label'] == 0 :
    df_gr.loc[idx, 'pred_label_3label'] = 1
  elif df_gr.loc[idx, 'pred_label'] == 4 :
    df_gr.loc[idx, 'pred_label_3label'] = 3
  else :
    df_gr.loc[idx, 'pred_label_3label'] = int(df_gr.loc[idx, 'pred_label'])

df_gr['pred_label_3label'] = df_gr['pred_label_3label'].apply(lambda x : int(x))

for idx in df_gr.index :
  if df_gr.loc[idx, 'target_3label'] == df_gr.loc[idx, 'pred_label_3label']:
    temp_eval = True
  else :
    temp_eval = False
  df_gr.loc[idx, 'eval_3'] = temp_eval

for idx in df_gr.index :
  if df_gr.loc[idx, 'targetpm'] == df_gr.loc[idx, 'pred_pm']:
    temp_eval = True
  else :
    temp_eval = False
  df_gr.loc[idx, 'eval_pm'] = temp_eval



print('5중분류 정확도 :', round(df_gr['eval_5'].sum() / len(df_gr), 2), '\n')
print('3중분류 정확도 :', round(df_gr['eval_3'].sum() / len(df_gr), 2), '\n')
print('이진분류 정확도 :', round(df_gr['eval_pm'].sum() / len(df_gr), 2), '\n')

df_gr.head()

5중분류 정확도 : 0.19 

3중분류 정확도 : 0.28 

이진분류 정확도 : 0.38 



,date,target,pred,targetpm,cnt_pred,pred_label,pred_pm,eval_5,target_3label,pred_label_3label,eval_3,eval_pm
0,2017-12-12,2,1.0,3.0,1.0,1,1,False,2.0,1,False,False
1,2018-01-12,1,1.0,1.0,1.0,1,1,True,1.0,1,True,True
2,2018-01-31,3,1.0,3.0,1.0,1,1,False,3.0,1,False,False
3,2018-04-20,2,1.0,3.0,1.0,1,1,False,2.0,1,False,False
4,2018-05-04,2,1.0,1.0,1.0,1,1,False,2.0,1,False,True


In [30]:
test_data3['pred'].value_counts()

1    3933
Name: pred, dtype: int64

In [31]:
df_gr['pred_label'].value_counts()

1    117
Name: pred_label, dtype: int64

In [33]:
df_gr['target'].value_counts()

2    44
3    25
1    22
4    15
0    11
Name: target, dtype: int64

In [34]:
df_gr['cnt_pred'].mean()

1.0

In [35]:
df_gr['eval_pm'].value_counts(normalize=True)

False    0.623932
True     0.376068
Name: eval_pm, dtype: float64